# Timings

This assignment is meant to be run on **four nodes** with 28 cores.

One thing we are trying to do is gauge the performance of a good MPI implementation and compare it to a good OpenMP implementation.  In order to do this, we will have to load some slightly different modules.

In [2]:
module unload cse6230
module load cse6230/gcc-omp-gpu
module list

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |
Currently Loaded Modulefiles:
  1) curl/7.42.1
  2) git/2.13.4
  3) python/3.6
  4) /nv/coc-ice/tisaac3/opt/pace-ice/modulefiles/jupyter/1.0
  5) cuda/9.1
  6) /nv/

In [4]:
module load cse6230

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |


([mvapich](http://mvapich.cse.ohio-state.edu/) is a fork of the [mpich](https://www.mpich.org/) MPI implementation with some modifications for performance on various HPC hardware)

## Measuring MPI primitives

**Task 1 (3 pts)** The file `benchmarks.c` includes some basic benchmarks for measuring the performance of various MPI point-to-point and collective operations.  Right now, it is incomplete!

p2p pingpong / broadcast pingpong

In [1]:
grep "TODO" benchmarks.c

  /* TODO: split the communicator `comm` into one communicator for ranks
  /* TODO: destroy the subcommunicator created in `splitCommunicator` */
  /* TODO: Record the MPI walltime in `tic_p` */
  /* TODO: Get the elapsed MPI walltime since `tic_in`,
  /* TODO: take the times from all processes and compute the maximum,


Refering to a good [MPI Tutorial](https://computing.llnl.gov/tutorials/mpi/) or
[lecture notes](http://vuduc.org/cse6230/slides/cse6230-fa14--06-mpi.pdf) as needed,
fill in the missing MPI routines.

Once you have done that, run the following script to generate a graph of benchmark bandwidths of MPI routines.  Note that these values are only for MPI messages within a node: values may be different when we start using multiple nodes. 

In [3]:
# please ignore the output here, I rerun with other nodes and have output in plot-input file
make clean
make
mpirun -n 56 ./benchmarks

rm -f *.o benchmarks
mpicc -fopenmp -I../../utils -g -Wall -O3 -std=c99 -c -o benchmarks.o benchmarks.c
mpicc -fopenmp -o benchmarks benchmarks.o 
MPI Version: 3.1
MVAPICH2 Version      :	2.3.2
MVAPICH2 Release date :	Fri August 9 22:00:00 EST 2019
MVAPICH2 Device       :	ch3:mrail
MVAPICH2 configure    :	--prefix=/nv/coc-ice/tisaac3/opt/pace-ice/mvapich2/2.3 --with-hwloc --with-pbs=/opt/torque/current --enable-romio --with-file-system=ufs+nfs --enable-shared --enable-sharedlibs=gcc
MVAPICH2 CC           :	gcc    -DNDEBUG -DNVALGRIND -O2
MVAPICH2 CXX          :	g++   -DNDEBUG -DNVALGRIND -O2
MVAPICH2 F77          :	gfortran -L/lib -L/lib   -O2
MVAPICH2 FC           :	gfortran   -O2

MPI # Procs: 56
MPI Wtime 1.57125e+09, precision 1e-06
MPI Wtime is global
MPI proc 0 host: rich133-k40-22-l.pace.gatech.edu
MPI proc 1 host: rich133-k40-22-l.pace.gatech.edu
MPI proc 2 host: rich133-k40-22-l.pace.gatech.edu
MPI proc 3 host: rich133-k40-22-l.pace.gatech.edu
MPI proc 4 host: rich133-k40-22-l

           14              8            208   +2.74301e-06   +7.58292e+07
           14             64           1664   +4.14610e-06   +4.01341e+08
           14            512          13312   +7.24006e-06   +1.83866e+09
           14           4096         106496   +1.66779e-05   +6.38547e+09
           14          32768         851968   +1.07024e-04   +7.96054e+09
           14         262144        6815744   +7.29103e-04   +9.34812e+09
           14        2097152       54525952   +3.41217e-03   +1.59798e+10
           28              8            432   +6.17099e-06   +7.00050e+07
           28             64           3456   +7.46918e-06   +4.62702e+08
           28            512          27648   +1.57001e-05   +1.76101e+09
           28           4096         221184   +1.15468e-04   +1.91554e+09
           28          32768        1769472   +1.13918e-04   +1.55328e+10
           28         262144       14155776   +7.16455e-04   +1.97581e+10
           28        2097152      1132

: 255

In [ ]:
make runbenchmarks

In [ ]:
display < benchmarks.png

(Right now the graph is showing no values because the "timing" values are negative until you complete the code)

**Task 2 (2 pts)** We've talked in class about a simplified model of the cost of an MPI message: $\lambda + g b$, where $\lambda$ is the latency and $g$ is the inverse bandwidth.
Using your graph for Send/Recv bandwidths for different message sizes (which was simply calculated from dividing the message size by the message time), estimate $\lambda$ (units secs) and $g$ (units secs/byte) for this MPI implementation on these nodes.

![](benchmarks.png)

![](benchmarks2.png)


| CommunicationType | g | $\lambda$ |
|---|---|---|
| 'Send'| 2.22926832e-11| 4.79391442e-07|
|'Bcast'|  4.22399938e-11| -4.86092968e-07|
|'Allreduce'| 1.23547847e-11| 2.85296871e-07|
|'Allgather'| 8.98445459e-12| -2.20375856e-09|
|'Alltoall'| 3.26118493e-11| 4.52594649e-08|


**Task 4 (3 pts)** The point-to-point sends and receives in MPI are _symmetric_: there must be a receive for each send.  What if we start with a list of messages that is _asymmetric_: we know who to send to, but not who to receive from.

Suppose rank $i$ has $N_i$ messages bound for receivers $r_{i,j}$ for $1 \leq j \leq N_i$.  Let $S_{i,j}$ be the size of the message from $i$ to $r_{i,j}$, and let $S_i = \sum_j S_{i,j}$ be the total outgoing message volume from rank $i$.

Process $r_{i,j}$ does not know it is going to receive a message from $i$.

Give pseudocode below for two algorithms, using MPI sends, receives, and collectives that we have talked about, to send all of the messages.

In the first one, assume that there is a large volume of communication: that $S_i \in O(P)$ for each $i$.

In the second one, assume that there is a small volume of communication, and a small number of communicators:
$S_i \in O(1)$, and each rank needs to send or receive a message from $O(1)$ other processes.

Polynomial Message: (indexing is 1-based)
```
MPI_init()
comm = communicator
int commSize = size of communicator
int rank = rank of current process
int buffer[commSize] = {-1}
for i = 1..commSize {
    if rank == i {
        buffer[i] = 0
    }
    if S[i,j] > 0 {
        buffer[j] = encode(S[i,j], r[i,j]) // any possible way to save 2 piece data in one integer
    }
}
MPI_Alltoall(MPI_IN_PLACE, 0, MPI_BYTE, buffer, numBytes, MPI_INT, comm);
for i = 1..N[i]{
    MPI_Send(message, S[i,j], MESSAGE_TYPE, r[i,j], i << 2 + j, comm) // any possible unique encoding of i,j
}
all_messages = Vector()
for i = 1..commSize {
    if buffer[i] != -1 {
        message_size, from = decode(buffer[j])
        int receive_buffer[message_size]
        // tag here corresponds to tag in send, (from, i) is (i, j)
        MPI_Recv(receive_buffer, message_size, MESSAGE_TYPE, from, from << 2 + i, comm)
        all_messages.push(receive_buffer)
    } 
}
deal with all_messages
```

Constant Message:
```
MPI_init()
comm = communicator
int commSize = size of communicator
int rank = rank of current process
int maxnumMessage
MPI_Allreduce(N[i], maxnumMessage, MPI_INT, MPI_MAX, comm)

int buffer[maxnumMessage]
for i == 1..maxnumMessage {
    buffer[i] = encode(S[rank, i], r[rank, i])
}
int* tmp_buffer
if (!rank){
    tmp_buffer = malloc(int*, maxnumMessage * commSize)
}
MPI_Gather(buffer, maxnumMessage, MPI_INT, tmp_buffer, maxnumMessage, MPI_INT, 0, comm)
process_buffer(tmp_buffer)
// process so that buffer for each comm contains who to receive from
// here we assume #receive_from won't exceed maxnumMessage, if it did, scatter a new integer will fix it.
MPI_Scatter(tmp_buffer, maxnumMessage, MPI_INT, buffer, maxnumMessage, MPI_INT, 0, comm)
for i = 1..N[i]{
    MPI_Send(message, S[i,j], MESSAGE_TYPE, r[i,j], i << 2 + j, comm) // any possible unique encoding of i,j
}
all_messages = Vector()
for i = 1..maxnumMessage{
    if (buffer[i] > 0){
        message_size, from = decode(buffer[j])
        int receive_buffer[message_size]
        // tag here corresponds to tag in send, (from, i) is (i, j)
        MPI_Recv(receive_buffer, message_size, MESSAGE_TYPE, from, from << 2 + i, comm)
        all_messages.push(receive_buffer)
    }
}

```